# Set up

## Install

In [1]:
!nvidia-smi

Wed Feb  5 16:22:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-PCIE-40GB          On  |   00000000:25:00.0 Off |                    0 |
| N/A   35C    P0             47W /  250W |       1MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [2]:
import os
import torch
import yaml
import shutil
import zipfile
import ultralytics
import random
import requests
import cv2
import itertools
import json

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch  # For PyTorch
print(torch.cuda.is_available())  # True means a GPU is accessible

from ultralytics import YOLO
from ultralytics import settings
ultralytics.checks()

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
Setup complete ✅ (64 CPUs, 503.2 GB RAM, 0.0/0.0 GB disk)


In [3]:
import subprocess

def install_opencv_headless():
    try:
        subprocess.run(["pip", "install", "opencv-python-headless"], check=True)
        print("opencv-python-headless installed successfully.")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")

install_opencv_headless()

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
opencv-python-headless installed successfully.



[notice] A new release of pip is available: 23.2.1 -> 25.0
[notice] To update, run: python -m pip install --upgrade pip


## Ultralytics settings

In [4]:
# refer to the following page
# https://docs.ultralytics.com/quickstart/#modifying-settings

In [5]:
!pwd

/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code/train


In [6]:
base_dir = "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc"
base_dir

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc'

In [7]:
#print(settings)

In [8]:
# refer to the following page
# https://docs.ultralytics.com/quickstart/#modifying-settings

# Update a setting+
settings.update({"datasets_dir": os.path.join(base_dir, "cvat_export")})
settings.update({"weights_dir": os.path.join(base_dir, "current_training/weights")})
settings.update({"runs_dir": os.path.join(base_dir, "current_training/runs")})
print(settings)

# Reset settings to default values
# settings.reset()

JSONDict("/home/hice1/jlee4338/.config/Ultralytics/settings.json"):
{
  "settings_version": "0.0.6",
  "datasets_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export",
  "weights_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/weights",
  "runs_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs",
  "uuid": "03e9e516df57c4506ba2f666b65d3a494d7d5521dfb3553eec5e6c3ca7475924",
  "sync": true,
  "api_key": "",
  "openai_api_key": "",
  "clearml": true,
  "comet": true,
  "dvc": true,
  "hub": true,
  "mlflow": true,
  "neptune": true,
  "raytune": true,
  "tensorboard": true,
  "wandb": false,
  "vscode_msg": true
}


## Import datasets

In [9]:
os.getcwd()

'/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/code/train'

In [ ]:
!ls ../../cvat_export

In [ ]:
base_dir = "../../cvat_export"

In [ ]:
# Path to the ZIP file
zip_file_path = os.path.join(base_dir, "cvat_0129_processed.zip")
extract_to = os.path.join(base_dir, "cvat_0129_processed_unzipped")
# Open the ZIP file and extract its contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)
print(f"Files extracted to: {extract_to}")

In [ ]:
!ls ../../cvat_export/cvat_0129_processed_unzipped

## Change .yaml files if needed

In [ ]:
# always start the path from the "datasets_dir": "/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export"
# path: ./export_0122_0921_processed_unzipped
# train: ./images/train  
# val: ./images/val     

In [ ]:
import yaml

In [ ]:
base_dir = "../../cvat_export/cvat_0129_processed_unzipped"

In [ ]:
# Filepath to your YAML file
yaml_files = [os.path.join(base_dir, "data_nc_1.yaml"),
              os.path.join(base_dir, "data_nc_5.yaml"),
              os.path.join(base_dir, "data_nc_6.yaml")]

for yaml_file in yaml_files:
    
    # Load the YAML file
    with open(yaml_file, "r") as file:
        data = yaml.safe_load(file)
        print(data["path"])
        data["path"] = "./cvat_0129_processed_unzipped"
        print(data["path"])
    
    # Write the changes back to the YAML file
    with open(yaml_file, "w") as file:
        yaml.dump(data, file, default_flow_style=False)

    print("Updated .yaml path")

# Check yolo pretrained model structure

In [ ]:
model = YOLO('yolo11m.pt')

In [ ]:
len(model.model.model)

In [ ]:
!find / -type f \( -name "*.yaml" -o -name "*.yml" \)

In [ ]:
#!find / -type f \( -name "*.yaml" -o -name "*.yml" \)
yaml_file = "/home/hice1/jlee4338/.local/lib/python3.10/site-packages/ultralytics/cfg/models/11/yolo11.yaml"

with open(yaml_file, "r", encoding="utf-8") as file:
    model_config = yaml.safe_load(file)

model_config

In [ ]:
model_config['nc']

In [ ]:
#print(len(model_config['backbone']))
#model_config['backbone']

In [ ]:
#print(len(model_config['head']))
#model_config['head']

# Only-detection model (nc=1)

## Reorganize the label folder

In [16]:
base_dir = "../../cvat_export/cvat_0129_processed_unzipped"

In [17]:
folder = os.path.join(base_dir, "labels")
try:
    shutil.rmtree(folder)
    print(f"Removed empty folder: {folder}")
except Exception as e:
    print(f"An error occurred: {e}")

Removed empty folder: ../../cvat_export/cvat_0129_processed_unzipped/labels


In [18]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels_nc_1")
target_folder = os.path.join(base_dir, "labels")

# Check if the target folder already exists
if os.path.exists(target_folder):
    print(f"Target folder '{target_folder}' already exists. Exiting.")
else:
    # Copy the entire folder structure
    shutil.copytree(current_folder, target_folder)
    print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

Copied folder from '../../cvat_export/cvat_0129_processed_unzipped/labels_nc_1' to '../../cvat_export/cvat_0129_processed_unzipped/labels'.


## Train by freeze_0_10 (with 23rd DFL frozen)

In [ ]:
base_dir = "../../cvat_export/cvat_0129_processed_unzipped"

In [ ]:
model_nc_1 = YOLO('yolo11m.pt')

In [ ]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_1.yaml")
yaml_file

In [ ]:
model_nc_1.train(data=yaml_file, freeze=11, epochs=15, batch=32, imgsz=640, workers=2,
            name="model_nc_1")

In [ ]:
model_nc_1.val(data=yaml_file, imgsz=640)

In [ ]:
model_nc_1.export(format="onnx")

## Validate on test set
always start from the beginning)

In [10]:
model_nc_1 = YOLO('/storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/model_nc_1/weights/best.pt')

In [20]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_1.yaml")
yaml_file

'../../cvat_export/cvat_0129_processed_unzipped/data_nc_1.yaml'

In [21]:
model_nc_1.val(data=yaml_file, imgsz=640, split="test")

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)


val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test... 130 images, 34 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 2780.40it/s]

val: New cache created: /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test.cache



/home/hice1/jlee4338/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  8.13it/s]


                   all        130        113      0.939      0.965       0.98      0.778
Speed: 0.5ms preprocess, 4.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/val


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15537fbe15a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

# 5-class-at-once model (nc=5)

In [25]:
# Restart the kernel
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=memory.free --format=csv,nounits

memory.free [MiB]
39452


## Reorganize the label folder

In [22]:
base_dir = "../../cvat_export/cvat_0129_processed_unzipped"

In [23]:
folder = os.path.join(base_dir, "labels")
try:
    shutil.rmtree(folder)
    print(f"Removed empty folder: {folder}")
except Exception as e:
    print(f"An error occurred: {e}")

Removed empty folder: ../../cvat_export/cvat_0129_processed_unzipped/labels


In [24]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels_nc_5")
target_folder = os.path.join(base_dir, "labels")

# Check if the target folder already exists
if os.path.exists(target_folder):
    print(f"Target folder '{target_folder}' already exists. Exiting.")
else:
    # Copy the entire folder structure
    shutil.copytree(current_folder, target_folder)
    print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

Copied folder from '../../cvat_export/cvat_0129_processed_unzipped/labels_nc_5' to '../../cvat_export/cvat_0129_processed_unzipped/labels'.


## Train by freeze_0_10 (with 23rd DFL frozen)

In [ ]:
model_nc_5 = YOLO('yolo11m.pt')

In [ ]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")
yaml_file

In [ ]:
model_nc_5.train(data=yaml_file, freeze=11, epochs=15, batch=32, imgsz=640, workers=4,
                 name="model_nc_5")

In [ ]:
model_nc_5.val(data=yaml_file, imgsz=640, line_width=10)

In [ ]:
model_nc_5.export(format="onnx")

## Validate on test set
always start from the beginning)

In [28]:
model_nc_5 = YOLO("../../current_training/runs/detect/model_nc_5/weights/best.pt")

In [29]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_5.yaml")
yaml_file

'../../cvat_export/cvat_0129_processed_unzipped/data_nc_5.yaml'

In [30]:
model_nc_5.val(data=yaml_file, imgsz=640, split="test")

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11m summary (fused): 303 layers, 20,033,887 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test... 130 images, 34 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 2760.60it/s]

val: New cache created: /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test.cache



/home/hice1/jlee4338/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  8.00it/s]


                   all        130        113      0.854      0.915      0.948      0.785
            prohibAnyT         35         38      0.899      0.789      0.923      0.737
           prohibSomeT         35         35      0.798      0.857      0.909      0.767
         prohibBusStop         24         24      0.882      0.958      0.948      0.749
           prohibClean         10         10      0.837          1      0.986      0.837
                 allow          6          6      0.853      0.971      0.972      0.837
Speed: 0.7ms preprocess, 3.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/val2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x15537c3282b0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
        

# 6-class-at-once model (nc=6)

In [31]:
torch.cuda.empty_cache()
!nvidia-smi --query-gpu=memory.free --format=csv,nounits

memory.free [MiB]
39312


## Reorganize the label folder

In [32]:
base_dir = "../../cvat_export/cvat_0129_processed_unzipped"

In [33]:
folder = os.path.join(base_dir, "labels")
try:
    shutil.rmtree(folder)
    print(f"Removed empty folder: {folder}")
except Exception as e:
    print(f"An error occurred: {e}")

Removed empty folder: ../../cvat_export/cvat_0129_processed_unzipped/labels


In [34]:
# Define the current folder paths and the target folder
current_folder = os.path.join(base_dir, "labels_nc_6")
target_folder = os.path.join(base_dir, "labels")

# Check if the target folder already exists
if os.path.exists(target_folder):
    print(f"Target folder '{target_folder}' already exists. Exiting.")
else:
    # Copy the entire folder structure
    shutil.copytree(current_folder, target_folder)
    print(f"Copied folder from '{current_folder}' to '{target_folder}'.")

Copied folder from '../../cvat_export/cvat_0129_processed_unzipped/labels_nc_6' to '../../cvat_export/cvat_0129_processed_unzipped/labels'.


## Train by freeze_0_10 (with 23rd DFL frozen)

In [ ]:
model_nc_6 = YOLO('yolo11m.pt')

In [ ]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_6.yaml")
yaml_file

In [ ]:
model_nc_6.train(data=yaml_file, freeze=11, epochs=15, batch=32, imgsz=640, workers=4,
                 name="model_nc_6")

In [ ]:
model_nc_6.val(data=yaml_file, imgsz=640, line_width=10)

In [ ]:
model_nc_6.export(format="onnx")

## Validate on test set
always start from the beginning)

In [35]:
model_nc_6 = YOLO("../../current_training/runs/detect/model_nc_6/weights/best.pt")

In [36]:
# where is .yaml file?
yaml_file = os.path.join(base_dir, "data_nc_6.yaml")
yaml_file

'../../cvat_export/cvat_0129_processed_unzipped/data_nc_6.yaml'

In [37]:
model_nc_6.val(data=yaml_file, imgsz=640, split="test")

Ultralytics 8.3.66 🚀 Python-3.10.12 torch-2.5.0+cu124 CUDA:0 (NVIDIA A100-PCIE-40GB, 40446MiB)
YOLO11m summary (fused): 303 layers, 20,034,658 parameters, 0 gradients, 67.7 GFLOPs


val: Scanning /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test... 130 images, 34 backgrounds, 0 corrupt: 100%|██████████| 130/130 [00:00<00:00, 2698.97it/s]

val: New cache created: /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/cvat_export/cvat_0129_processed_unzipped/labels/test.cache



/home/hice1/jlee4338/.local/lib/python3.10/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:01<00:00,  8.18it/s]


                   all        130        113      0.865      0.929      0.964       0.81
            prohibAnyT         35         38      0.858      0.842      0.947      0.765
           prohibHalfD         21         21      0.863      0.898      0.943      0.798
           prohibFullD         14         14      0.706      0.929       0.94        0.8
         prohibBusStop         24         24      0.881      0.924      0.972      0.774
           prohibClean         10         10      0.886          1      0.986      0.857
                 allow          6          6          1      0.981      0.995      0.866
Speed: 0.6ms preprocess, 3.9ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /storage/ice1/2/4/jlee4338/ongoing/park_sign/nyc/current_training/runs/detect/val3


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x1553625930a0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
     